# Chapter 2.1

### Exercise 2.1

**Define a better version of `make-rat` that handles both positive and negative arguments. `make-rat` should normalize the sign so that if the rational number is positive, both the numerator and denominator are positive, and if the rational number is negative, only the numerator is negative.**

In [1]:
(define (gcd _x _y)
  (define x (abs _x))
  (define y (abs _y))
  (cond ((= y 0) x)
        ((< x y) (gcd y x))
        (else (gcd y (modulo x y)))))
    
(define (make-rat n d)
  (define g (gcd n d ))
  (define gs (if (< d 0) (- g) g))
  (cons (/ n gs) (/ d gs)))
  
(define (numer rat) (car rat))
(define (denom rat) (cdr rat))
  
(define (add-rat a b)
  (make-rat (+ (* (numer a) (denom b)) (* (numer b) (denom a))) (* (denom a) (denom b))))
  
(define (sub-rat a b)
  (make-rat (- (* (numer a) (denom b)) (* (numer b) (denom a))) (* (denom a) (denom b))))

(define (mul-rat a b)
  (make-rat (* (numer a) (numer b)) (* (denom a) (denom b))))
  
(define (div-rat a b)
  (make-rat (* (numer a) (denom b)) (* (denom a) (numer b))))
  
(define (equal-rat? a b)
  (= (* (numer a) (denom b)) (* (numer b) (denom a))))
  
(define (print-rat a)
  (display (numer a))
  (display "/")
  (display (denom a))
  (newline))

In [2]:
(print-rat (make-rat 1 2))
(print-rat (make-rat 2 -6))
(print-rat (make-rat -3 -4))
(print-rat (make-rat -1 2))

1/2
-1/3
3/4
-1/2


### Exercise 2.2

**Consider the problem of representing line segments in a plane. Each segment is represented as a pair of points: a starting point and an ending point. Deﬁne a constructor `make-segment` and selectors `start-segment` and
`end-segment` that deﬁne the representation of segments in terms of points. Furthermore, a point can be represented as a pair of numbers: the x coordinate and the y coordinate. Accordingly, specify a constructor `make-point` and selectors `x-point` and `y-point` that deﬁne this representation. Finally, using your selectors and constructors, deﬁne a procedure `midpoint-segment` that takes a line segment as argument and returns its midpoint (the point whose coordinates are the average of the coordinates of the endpoints). To try your procedures, you’ll need a way to print points:**

```
(define (print-point p)
    (newline)
    (display "(")
    (display (x-point p))
    (display ",")
    (display (y-point p))
    (display ")"))
```

In [3]:
(define (make-point x y) (cons x y))
(define (x-point p) (car p))
(define (y-point p) (cdr p))
(define (print-point p)
    (newline)
    (display "(")
    (display (x-point p))
    (display ",")
    (display (y-point p))
    (display ")"))

(define (make-segment s e) (cons s e))
(define (start-segment l) (car l))
(define (end-segment l) (cdr l))

(define (midpoint-segment ls)
  (define (avg a b) (/ (+ a b) 2))
  (make-point (avg (x-point (start-segment ls)) (x-point (end-segment ls)))
              (avg (y-point (start-segment ls)) (y-point (end-segment ls)))))

In [4]:
(define a (make-point 1 2))
(define b (make-point 3 6))
(define ab (make-segment a b))
(print-point (midpoint-segment ab))


(2,4)

### Exercise 2.3

**Implement a representation for rectangles in a plane. (Hint: You may want to make use of Exercise 2.2.) In terms of your constructors and selectors, create procedures that compute the perimeter and the area of a given rectangle. Now implement a diﬀerent representation for rectangles. Can you design your system with suitable abstraction barriers, so that the same perimeter and area procedures will work using either representation?**

Below we implement a representation for rectangles where they are defined by their diagonally opposite points. We also define a selector called `side-lengths` which evaluates to the pair of side lengths of the given rectangle. `perimeter` and `area` are defined in terms of `side-lengths`.

In [5]:
(define (make-rectangle bottom-left top-right)
  (cond ((> (x-point bottom-left) (x-point top-right)) (make-rectangle top-right bottom-left))
        ((> (y-point bottom-left) (y-point top-right))
         (make-rectangle (make-point (x-point bottom-left) (y-point top-right))
                         (make-point (x-point top-right) (y-point bottom-left))))
        (else (cons bottom-left top-right))))

(define (side-lengths rectangle)
  (define bottom-left (car rectangle))
  (define top-right (cdr rectangle))
  (cons (abs (- (x-point bottom-left) (x-point top-right))) (abs (- (y-point bottom-left) (y-point top-right)))))

(define (perimeter rectangle)
  (define sl (side-lengths rectangle))
  (* 2 (+ (car sl) (cdr sl))))

(define (area rectangle)
  (define sl (side-lengths rectangle))
  (* (car sl) (cdr sl)))

In [6]:
(perimeter (make-rectangle (make-point 1 1) (make-point 3 5)))

12

In [7]:
(perimeter (make-rectangle (make-point 2 0) (make-point 0 2)))

8

In [8]:
(area (make-rectangle (make-point 4 5) (make-point 7 7)))

6

Now we change the representation of rectangles by modifying `make-rectangle` and `side-lengths`. This time a rectangle is represented in terms of a center and its two side lengths.

In [9]:
(define (make-rectangle center width height)
  (cons center (cons width height)))

(define (side-lengths rectangle)
  (cdr rectangle))

In [10]:
(area (make-rectangle (make-point 0 0) 4 5))

20

In [11]:
(perimeter (make-rectangle (make-point 1 3) 3 2))

10

### Exercise 2.4

**Here is an alternative procedural representation of pairs. For this representation, verify that `(car (cons x y))` yields `x` for any objects `x` and `y`.**

In [12]:
(define (cons x y)
    (lambda (m) (m x y)))
(define (car z)
    (z (lambda (p q) p)))

**What is the corresponding deﬁnition of `cdr` ? (Hint: To verify that this works, make use of the substitution model of Section 1.1.5.)**

The corresponding definition of `cdr` is

In [13]:
(define (cdr z)
  (z (lambda (p q) q)))

The following examples show this works:

In [14]:
(cdr (cons 1 2))

2

In [15]:
(cdr (cons 123 48))

48

The following substitutions prove this as well:

```
(cdr (cons x y))
((cons x y) (lambda (p q) q))
((lambda (m) (m x y)) (lambda (p q) q))
((lambda (p q) q) x y)
y
```

NOTE: This is how pairs are defined in [Church Encoding](https://en.wikipedia.org/wiki/Church_encoding#Church_pairs).

### Exercise 2.5

**Show that we can represent pairs of nonnegative integers using only numbers and arithmetic operations if we represent the pair $a$ and $b$ as the integer that is the product $2^a3^b$. Give the corresponding deﬁnitions of the procedures `cons`, `car`, and `cdr`.**

To show that we can represent elements from one set (in this case, $\mathbb{Z_{0+}}\times\mathbb{Z}_+$) as members of another set (in this case, $\mathbb{Z}_+$), we need only to show that there exists a reversible function from the first set to the other.

That is, there exists an $f \in \mathbb{Z_{0+}}\times\mathbb{Z}_+ \rightarrow \mathbb{Z}_+$ and its inverse $f^{-1} \in \mathbb{Z}_+ \rightarrow \mathbb{Z}_{0+}\times\mathbb{Z}_+$ such that for all $x \in \mathbb{Z}_{0+}\times\mathbb{Z}_+$, $f^{-1}(f(x)) = x$.

The function $f$ is already given: $f(a, b) = 2^a3^b$ for the pair of non-negative integers $(a,b)$. We need to find the $f^{-1}$. At a glance, we can say that to get $a$ from $2^a3^b$, we need to find out how many times we can divide $2^a3^b$ with 2. Similarly with $b$. In more mathematically precise terms, we can specify $f^{-1}$ in the following way.

Take $x=2^a3^b$. Taking the base-2 logarithm of both sides gives us
$$
\log_2{x} = \log_2{2^a3^b} = a\log_2{2} + b\log_2{3} = a + 1.585 b
$$

Since $b >= 0$, we can say that $\log_2{x} >= a$, so $\lfloor\log_2{x}\rfloor >= a$. So if we take the GCD of $2^{\lfloor\log_2{x}\rfloor}$ and x, we will get $2^a$. So, to get $a$ from $x$, we can take $\log_2{\gcd(x, 2^{\lfloor\log_2{x}\rfloor})}$. Applying the same to obtain $b$ as well, we define $f^{-1}$ as the pair:

$$
f^{-1}(x) = (\log_2{\gcd(x, 2^{\lfloor\log_2{x}\rfloor})}, \log_3{\gcd(x, 3^{\lfloor\log_3{x}\rfloor})})
$$

To define $car$ and $cdr$ now, we translate the definition above Scheme, after defining some helper functions.

In [16]:
(define (gcd a b)
  (cond ((= b 0) a)
        ((> b a) (gcd b a))
        (else (gcd b (remainder a b)))))

(define (log-base base x) (/ (log x) (log base)))

Now, we define the new `cons`, `car` and `cdr` functions within a let block. The additional `round` in implementations of `cdr` and `car` is to recover from errors that get introduced due to limited floating point precision. The examples printed demonstrate that these definitions work as expected.

In [17]:
(let (
      (cons (lambda (a b) (* (expt 2 a) (expt 3 b))))
      (car (lambda (x) (round (log-base 2 (gcd x (expt 2 (floor (log-base 2 x))))))))
      (cdr (lambda (x) (round (log-base 3 (gcd x (expt 3 (floor (log-base 3 x))))))))
     )
  
  (let ((x1 (cons 15 20))
        (x2 (cons 0 0)))
    (display (car x1))
    (display ", ")
    (display (cdr x1))
    (newline)
    (display (car x2))
    (display ", ")
    (display (cdr x2))
    )
  )
  

15.0, 20.0
0, 0

## Exercise 2.6

**In case representing pairs as procedures wasn’t mind-boggling enough, consider that, in a language that can manipulate procedures, we can get by without numbers (at least insofar as nonnegative integers are concerned) by implementing 0 and the operation of adding 1 as**
```
(define zero (lambda (f) (lambda (x) x)))
(define (add-1 n)
  (lambda (f) (lambda (x) (f ((n f) x)))))
```  

**This representation is known as _Church numerals_, after its inventor, Alonzo Church, the logician who invented the $\lambda$-calculus.**

**Define one and two directly (not in terms of `zero` and `add-1`). (Hint: Use substitution to evaluate `(add-1 zero)`). Give a direct definition of the addition procedure `+` (not in terms of repeated application of `add-1`).**

We apply `add-1` to `zero` to evaluate `one` first. In the substitutions, primed variables (`x'`, `f'`) are used to avoid capturing a free variable of the same name.

```
(add-1 zero)
(add-1 (lambda (f) (lambda (x) x)))
(lambda (f) (lambda (x) (f (( (lambda (f') (lambda (x') x')) f) x)))))
(lambda (f) (lambda (x) (f ((lambda (x') x') x))))
(lambda (f) (lambda (x) (f x))) ; = one
```

Then, we can apply `add-1` to `one` to evaluate `two`:

```
(add-1 one)
(add-1 (lambda (f) (lambda (x) (f x))))
(lambda (f) (lambda (x) (f (((lambda (f') (lambda (x') (f' x'))) f) x))))
(lambda (f) (lambda (x) (f (((lambda (x') (f x')) x))))
(lambda (f) (lambda (x) (f (f x)))) ; = two
```

In plain English, the natural number $n$ is represented in Church numerals by a function that takes a function (say, $f$) as an argument, and returns a function that takes an argument (say, $x$) and returns the value that is the result of applying $f$ to $x$ $n$ times. Using this insight, the addition procedure can be defined as

```
(define + (lambda (m n) (lambda (f) (lambda (x) ((m f) (n f x))))))
```

The proof of this is below, in the notation of $\lambda$-calculus. I have used $f^nx$ to mean the application of $f$ on $x$ repeated $n$ times, i.e $f(f(f...f(x)...))$. Let's define $m$ and $n$ as the following so we have to find an expression for $\lambda{f}.\lambda{x}.f^{m+n}x$ as our answer. (_Note that $m$ and $n$ are used in two senses in the equation, where perhaps different symbols would have been better. As a superscript of $f$, they mean the natural number itself, i.e. in the sense to mean repeated application that many times. As variables in the lambda expressions, they denote the Church numeral corresponding to the natural number._)

$$
\begin{align}
m &= \lambda{f}.\lambda{x}.f^mx\\
n &= \lambda{f}.\lambda{x}.f^nx\\
\end{align}
$$

We need something like $f^mf^nx$ in the body of $m+n$. To achieve this, we will follow this strategy. a) Apply $m$ and $n$ on values to get rid of the bindings so we are just left with expressions containing free variabels, which will be a lot easier to manipulate. b) Massage the resulting expressions to get the function body we need, and c) Bind the right variables to obtain a function. To begin, let's apply $m$ to $g$.

$$
\begin{align}
mg &= (\lambda{f}.\lambda{x}.f^mx)g\\
   &= \lambda{x}.g^mx
   \end{align}
   $$
   
Now, let's apply $n$ to $g$ and then to $z$.
$$
\begin{align}
ngz &= (\lambda{f}.\lambda{x}.f^nx)gz\\
&= (\lambda{x}.g^nx)z\\
&= g^nz
\end{align}
$$

At a glance, it looks like we could get what we need if we a) replace $x$ in $g^mx$ with $g^nz$, b) abstract out $z$ as a parameter, then c) abstract out $g$. We do step a) and b) below, starting with applying $ngz = g^nz$ to $mg =  \lambda{x}.g^mx$ and abstracting out $z$.

$$
\begin{align}
(mg)(ngz) &= (\lambda{x}.g^mx)(g^nz)\\
&= g^m(g^nz)\\
\lambda{z}.(mg)(ngz) &= \lambda{z}.g^m(g^nz) = \lambda{z}.g^{m+n}z
\end{align}
$$

Now, lets abstract out $g$ and we're almost done!

$$
\lambda{g}.\lambda{z}.(mg)(ngz) = \lambda{g}.\lambda{z}.g^m(h^nz)
$$

To make the notation more familiar, lets flip the equation and do $\alpha$-substitution by replacing $g$ with $f$ and $z$ with $x$.

$$
\lambda{f}.\lambda{x}.f^{m+n}x = \lambda{f}.\lambda{x}.(mf)(nfx)
$$

To test that this actually works, we can try it out in Scheme. For that, we'll need a way to translate Church numerals to regular numbers as they are represented in Scheme. This is very simple:  just pass 0 as the $x$ and a function that returns its input incremented by one as the $f$.

In [18]:
(define (church-to-regular n)
  (define (plus1 x) (+ x 1))
  (display ((n plus1 ) 0))
  (newline)
  )

Now we can move on to defining the church numeral for zero and a few other numbers, and the function `add-1`.

In [19]:
(define zero (lambda (f) (lambda (x) x)))
(define (add-1 n)
  (lambda (f) (lambda (x) (f ((n f) x)))))
(define one (add-1 zero))
(define two (add-1 one))
(define three (add-1 two))
(define four (add-1 three))
(define five (add-1 four))


We can test our addition function by defining it first, then testing it on the numbers declared above.

In [20]:
(define add (lambda (m n) (lambda (f) (lambda (x) ((m f) ((n f) x))))))
(church-to-regular zero)
(church-to-regular one)

0
1


In [21]:
(church-to-regular (add one two))
(church-to-regular (add zero two))
(church-to-regular (add two four))

3
2
6


The above demonstrate that the addition function really works with Church numerals.

## Exercise 2.7

**Alyssa's program is incomplete because she has not specified the implementation of the interval abstraction. Here is a definition of the interval constructor:**

In [22]:
(define (make-interval a b) (cons a b))

**Define selectors `upper-bound` and `lower-bound` to complete the implementation.**

Since the constructor was implemented as a pair, we can use `car` and `cdr` in the definitions of the selectors:

In [23]:
(define (upper-bound interval) (cdr interval))
(define (lower-bound interval) (car interval))

*Note: Including some definitions here that given in the text but are not in the execution environment yet, and some that haven't been mentioned but will be useful.*

In [24]:
(define (add-interval x y)
  (make-interval (+ (lower-bound x) (lower-bound y))
                 (+ (upper-bound x) (upper-bound y))))

(define (mul-interval x y)
  (let ((p1 (* (lower-bound x) (lower-bound y)))
        (p2 (* (lower-bound x) (upper-bound y)))
        (p3 (* (upper-bound x) (lower-bound y)))
        (p4 (* (upper-bound x) (upper-bound y))))
       (make-interval (min p1 p2 p3 p4)
                      (max p1 p2 p3 p4))))

(define (display-interval x)
  (display "(")
  (display (lower-bound x))
  (display ", ")
  (display (upper-bound x))
  (display ")")
  (newline))

## Exercise 2.8

**Using reasoning analogous to Alyssa's, describe how the difference of two intervals may be computed. Define a corresponding subtraction procedure, called `sub-interval`**.

Using reasoning analogous to that used in the definition of `add-interval`, we can say that difference of two intervals can be computed as the interval made of lower and upper bounds which are the differences of the lower and upper bounds of the original intervals. That is,

In [25]:
(define (sub-interval x y)
  (make-interval (- (lower-bound x) (lower-bound y))
                 (- (upper-bound x) (upper-bound y))))

## Exercise 2.9

**The _width_ of an interval is half of the difference between its upper and lower bounds. The width is a measure of the uncertainty of the number specified by the interval. For some arithmetic operations the width of the result of combining two intervals is a function only of the widths of the argument intervals, whereas for others the width of the combination is not a function of the widths of the argument intervals. Show that the width of the sum (or difference) of two intervals is a function of only the widths of the intervals being added (or subtracted). Give examples to show that this is not true for multiplication or division.**

If $x$ is an interval, let $x_l$ be its lower bound and let $x_u$ be its upper bound. We could call $x_w = x_u - x_l$ the width of the interval. If $z = x + y$, then we know from the definition that $z_l = x_l + y_l$ and $z_u = x_u + y_u$. So, the width of $z$ is $z_w = (x_u + y_u) - (x_l + y_l) = (x_u - x_l) + (y_u - y_l) = x_w + y_w$. This means the width of the sum is the sum of the width of the two intervals. Similarly, if we substitute addition with subtraction, i.e. $z = x - y$, we get $z_w = (x_u - y_u) - (x_l - y_l) = (x_u - x_l) - (y_u - y_l) = x_w - y_w$. This means the width of the difference is the differnce of the widths of the two intervals.

For multiplication, we can show that this is not true by taking two pairs of intervals with equal lenghts and multiplying them.

Let $a = (0, 1)$ and $b = (1, 2)$. Their product is $ a \dot b = (0, 2)$ according to the definition of multiplication. Now, take $c = (2,3)$ and compute the product $ b \dot c$, which comes out to be $(2, 6)$. Since $a, b$ and $c$ all had a width of $1$, but since $a \dot b$ and $b \dot c$ have different widths, it is evident that the width of the product of intervals is not a function solely of the widths of the intervals being multiplied.

Similarly, for division, the same set of intervals work as a counterexample. $\frac{a}{b}$ turns out to be $(0,1)$, which has a width of 1, and $\frac{b}{c}$ turns out to be $(\frac{1}{3}, 1)$, which has a width of $\frac{2}{3}$. Since $1 \neq \frac{2}{3}$, it is demonstrated that the width of the ratio of two intervals is not a function solely of the widths of the fractions being divided.

## Exercise 2.10

**Ben Bitdiddle, an expert systems programmer, looks over Alyssa’s shoulder and comments that it is not clear what it means to divide by an interval that spans zero. Modify Alyssa’s code to check for this condition and to signal an error if it occurs.**

By "an interval that spans zero", I assume that the question means to say an interval in which one (or both) of the bounds is zero, because it is the only interpretation that would lead to division not being defined.

We can define `div-interval` in the following way to check for this:

In [26]:
(define (div-interval x y)
  (if (or (= (upper-bound y) 0) (= (lower-bound y) 0))
    (error "Cannot divide by an interval spanning zero!")  
    (mul-interval
      x
     (make-interval (/ 1.0 (upper-bound y))
                    (/ 1.0 (lower-bound y))))))

In [27]:
(div-interval (make-interval 1 2) (make-interval 0 1))

Cannot divide by an interval spanning zero!
  context...:
   body of top-level


In [28]:
(display-interval (div-interval (make-interval 3 2) (make-interval 2 1)))

(1.0, 3.0)


## Exercise 2.11

**In passing, Ben also cryptically comments: "By testing the signs of the endpoints of the intervals, it is possible to break mul-interval into nine cases, only one of which requires more than two multiplications." Rewrite this procedure using Ben’s suggestion.**




There are three possible combinations for the signs of the ends of an interval: (-, -), (-, +) and (+, +), since (+, -) is not possible. This means we have $3^2 = 9$ possibilities for the sign combinations of the input. If we enumerate them and do some analysis as to what the maximum possible and minimum possible products of the components, are, we find that for all except one combination of the input signs, we already know which ends of the inputs to multiply to produce the output. The one combination that this doesn't work for is (-, +) and (-, +). If the intervals being multiplied are (a, b) and (c, d), then either $bc$ or $ad$ could be the lower bound of the interval and either $bd$ or $ac$ could be the upper bound of the interval, so four multiplications are required for this case. The following definition of `mul-interval-optimized` computes the product of two intervals in this optimized manner. 

(define (mul-interval-optimized x y)
  (define a (lower-bound x))
  (define b (upper-bound x))
  (define c (lower-bound y))
  (define d (upper-bound y))
  (define (neg n) (< n 0))
  (define (pos n) (>= n 0))
  (cond ((and (pos a) (pos b) (pos c) (pos d)) (make-interval (* a c) (* b d)))
        ((and (pos a) (pos b) (neg c) (pos d)) (make-interval (* b c) (* b d)))
        ((and (pos a) (pos b) (neg c) (neg d)) (make-interval (* b d) (* a c)))
        ((and (neg a) (neg b) (pos c) (pos d)) (make-interval (* a c) (* b c)))
        ((and (neg a) (neg b) (neg c) (pos d)) (make-interval (* a d) (* a c)))
        ((and (neg a) (neg b) (neg c) (neg d)) (make-interval (* a c) (* b d)))
        ((and (neg a) (pos b) (pos c) (pos d)) (make-interval (* a d) (* b d)))
        ((and (neg a) (pos b) (neg c) (neg d)) (make-interval (* b c) (* a c)))
        ((and (neg a) (pos b) (neg c) (pos d)) (make-interval (min (* b c) (* a d))
                                                              (max (* b d) (* a c))))
        ))

**After debugging her program, Alyssa shows it to a potential user, who complains that her program solves the wrong problem. He wants a program that can deal with numbers represented as a center value and an additive tolerance; for example, he wants to work with intervals such as 3.5 ± 0.15 rather than [3.35, 3.65]. Alyssa returns to her desk and fixes this problem by supplying an alternate constructor and alternate selectors:**



In [29]:
(define (make-center-width c w)
(make-interval (- c w) (+ c w)))
(define (center i)
(/ (+ (lower-bound i) (upper-bound i)) 2))
(define (width i)
(/ (- (upper-bound i) (lower-bound i)) 2))

**Unfortunately, most of Alyssa’s users are engineers. Real engineering situations usually involve measurements with only a small uncertainty, measured as the ratio of the width of the interval to the midpoint of the interval. Engineers usually specify percentage tolerances on the parameters of devices, as in the resistor specifications given earlier.**

## Exercise 2.12

**Define a constructor `make-center-percent`
that takes a center and a percentage tolerance and produces the desired interval. You must also define a selector
`percent` that produces the percentage tolerance for a given
interval. The `center` selector is the same as the one shown
above.**

In [30]:
(define (make-center-percent c p)
  (define width (* c (/ p 100)))
  (make-center-width c width))

(define (percent i)
  (define w (width i))
  (* (/ w (center i)) 100)) 

## Exercise 2.13

**Show that under the assumption of small
percentage tolerances there is a simple formula for the approximate percentage tolerance of the product of two intervals in terms of the tolerances of the factors. You may
simplify the problem by assuming that all numbers are positive.**

Let $(a \pm p\%)$ and $(b \pm q\%)$ be two intervals we are trying to multiply. Since we are assuming all numbers are positive, $(a - p\%)$ and $(b - q\%)$, the lower ends of both intervals, will multiply to produce the smallest product. So, the lower end of the product is  $(a-p\%)(b-q\%)$ and correspondingly the upper is $(a+p\%)(b+q\%)$. Using absolute values instead of percentages, lets say $r=\frac{pa}{100}$ and $s=\frac{qb}{100}$. Then the lower bound of the product is

$$
\begin{align}
$(a-p\%)(b-q\%)$ &= (a-r)(b-s)\\
&= ab - as -rb + rs\\
&= ab - a\frac{qb}{100} - \frac{pa}{100}b + rs\\
&= ab - (ab)\frac{q + p}{100} = rs\\
&\approx ab - (ab)\frac{q + p}{100}
\end{align}
$$

In the last step above, we ignore the $rs$ because we are assuming small percentage tolerances and the product of two small numbers less than one is an even smaller number.

Similarly, the upper bound of the product will be

$$
ab + (ab)\frac{q+p}{100}
$$

So, we can write the interval as $(ab - (p+q)\%, ab+(p+q)\%)$, or $ab \pm (p+q)\%$. This means the percentage tolerance of the product is the sum of the percentage tolerances of the intervals, which is indeed a simple formula.

**After considerable work, Alyssa P. Hacker delivers her finished system. Several years later, after she has forgotten all about it, she gets a frenzied call from an irate user, Lem E. Tweakit. It seems that Lem has noticed that the formula for parallel resistors can be written in two algebraically equivalent ways:**

$$
\frac{R_1R_2}{R_1 + R_2}
$$
**and**
$$
\frac{1}{1/R_1 + 1/R_2}
$$

**He has written the following two programs, each of which computes the parallel-resistors formula differently:**

In [31]:
(define (par1 r1 r2)
  (div-interval (mul-interval r1 r2)
                (add-interval r1 r2)))

(define (par2 r1 r2)
  (let ((one (make-interval 1 1)))
    (div-interval
     one (add-interval (div-interval one r1)
                       (div-interval one r2)))))

**Lem complains that Alyssa’s program gives different answers for the two ways of computing. This is a serious complaint.**

## Exercise 2.14

**Demonstrate that Lem is right. Investigate
the behavior of the system on a variety of arithmetic expressions. Make some intervals $A$ and $B$, and use them in
computing the expressions $A/A$ and $A/B$. You will get the
most insight by using intervals whose width is a small percentage of the center value. Examine the results of the computation in center-percent form (see Exercise 2.12).**

In [32]:
(define i1 (make-interval 1 2))
(define i2 (make-interval 2 3))

(display-interval (par1 i1 i2))
(display-interval (par2 i1 i2))


(0.4, 2.0)
(0.6666666666666666, 1.2000000000000002)


The above demonstrates that Lem is right. The two intervals are clearly not the same. Lets make some intervals and compute their $A/A$ and $A/B$ as instructed by the question.

In [33]:
(define A (make-interval 10 12))
(define B (make-interval 90 110))
(define C (make-interval 1 2))
(define D (make-interval 2 2))
(define E (make-interval 1 100))

(display-interval (div-interval A A))
(display-interval (div-interval A B))
(display-interval (div-interval C C))

(0.8333333333333333, 1.2000000000000002)
(0.09090909090909091, 0.13333333333333333)
(0.5, 2.0)


In [34]:
(display-interval (div-interval D D))
(display-interval (div-interval E E))
(display-interval (div-interval B B))

(1.0, 1.0)
(0.01, 100.0)
(0.8181818181818181, 1.2222222222222223)


In [35]:
(display-interval (div-interval C D))
(display-interval (div-interval B D))
(display-interval (div-interval B C))

(0.5, 1.0)
(45.0, 55.0)
(45.0, 110.0)


In [36]:
(display-interval (div-interval D C))
(display-interval (div-interval D B))

(1.0, 2.0)
(0.01818181818181818, 0.022222222222222223)


For expressions $A/A$, it clearly looks like the lower the percentage tolerance of $A$, the lower the percentage tolerance of $A/A$ will be. Also, it looks like the ends of the interval are always reciprocals of one another. So if we take the percentage tolerance higher, the ends of $A/A$, while always being reiprocal, keep going further from $[1,1]$.

The $A/B$ expressions are interesting as well. Dividing by a zero-tolerance interval seems to have the same effect as dividing the ends of the interval by the corresponding real number. So the tolerance of $A$ is preserved.

I think looking at these in the endpoint form makes it a lot harder to see the relationship, but it looks like if you divide intervals, the result is an interval with a midpoint that is the ratio of the midpoints of the two intervals and a tolerance that is about the sum of the intervals.

## Exercise 2.15
**Eva Lu Ator, another user, has also noticed
the different intervals computed by different but algebraically
equivalent expressions. She says that a formula to compute
with intervals using Alyssa’s system will produce tighter
error bounds if it can be written in such a form that no variable that represents an uncertain number is repeated. Thus,
she says, `par2` is a “better” program for parallel resistances
than `par1`. Is she right? Why?**

I think she is right. By taking the earlier example with `par1` and `par2`, we can observe that `par2` computed an answer with lower tolerance and with a midpoint closer to what the answer would have been if the tolerances were zero. This seems to be because multiplying and dividing two intervals seems to produce a result with their tolerances added (at least when the percentage tolerances are small), so it seems best to keep the interval-interval operations to minimum. 

## Exercise 2.16

**Explain, in general, why equivalent algebraic expressions may lead to different answers. Can you devise an interval-arithmetic package that does not have this shortcoming, or is this task impossible? (Warning: This problem is very difficult.)**

I am fairly certian this is not possible. Here's why. When we talk about 'equivalent algebraic expressions', we need a strict definition for what counts as an algebraic operation and what counts as algebraic. With regular real numbers, these are defined by the field axioms and the theorems resulting from it, since the real numbers are a field. When we take algebraic expressions that are often used with real numbers and say two of them are equivalent, it can only be extended to other fields, and in general not other types of algebraic structures. For example, matrices and matrix multiplication don't form a field, so many algebraic expressions are not even defined for them, let alone "equivalent" expressions producing equivalent results.

So the question we must ask is, are intervals a field? I think they aren't. Some of the field axioms of interest are the existence of a multiplicative and additive identities and corresponding inverses. We can think of $[0, 0]$ as the additive identity and $[1,1]$ as the multiplicative one. The definition of the additive identity certainly fits with the definition of addition in the book. But the field axioms also state that each element that is not the additive identity must have a multiplicative inverse. Now, take $[a,b]$, whose "inverse" under our definition is $[1/b, 1/a]$. Multiplying them, we get $[a/b, b/a]$, which except in the special case of $a=b$ is not equal to our multiplicative identity.

So under the given definitions of the operations on intervals, there can be no interval-arithmetic package that does not have this shortcoming.